# Initialization

In [ ]:
# importing relevant files and packages
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics, GLPK, XLSX, Plots, LaTeXStrings
WAF_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_WAF.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
WAF_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_WAF.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))

# commodities from source to terminal node
commodities = []
for i in 1:size(WAF_demand, 1)
    push!(commodities, string(WAF_demand.Origin[i], "s", "|", WAF_demand.Destination[i], "t"))
end
;

# Services

The following scenarios are run one by one

In [ ]:
# services for low scenario
data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/services/WAF_0_services_4_0.csv", delim=',', stringtype = String))
df = data[1:end,5:end];

In [ ]:
# services for mid scenario
data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/services/WAF_1_services_4_0.csv", delim=',', stringtype = String))
df = data[1:end,5:end];

In [ ]:
# services for high scenario
data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/services/WAF_2_services_4_0.csv", delim=',', stringtype = String))
df = data[1:end,5:end];

In [ ]:
# max lenght of service dataframes
max_length = maximum(length, eachrow(df[:, 1:end]))

# initializing datastructure for services
services = Vector{Matrix{Any}}(undef, size(df, 1))
for i in 1:size(df, 1)
    row_data = [get(df[i, :], j, missing) for j in 1:max_length + 1]
    non_missing_data = filter(x -> !ismissing(x), row_data)
    services[i] = hcat(non_missing_data...)
end

# unique ports of instance
service_ports = []
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports)
;

# Port modifications

In [ ]:
# list of ports that are visited by multiple services
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j))

# adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i))
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes)

# source and terminal ports adjacency list (subfigure (a) of figure 15)
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# transshipment nodes adjacency list (subfigure (c) of figure 15)
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])])
    end
end

# adjacency list of services
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service])
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0])
    end
end

# penalty edges adjacency list (subfigure (b) of figure 15)
penalty_adj = []
for k in 1:size(WAF_demand,1)
    # adding revenue that is later subtracted to equalize it for model conveniency
    push!(penalty_adj, [string(WAF_demand.Origin[k], "s"), string(WAF_demand.Destination[k], "t"), 1000 + filter(row -> row.Origin == string(WAF_demand.Origin[k]) && row.Destination == string(WAF_demand.Destination[k]), WAF_demand)[1,4]])
end

# dummy edges to solve Dijkstra's shortest path
dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end

# final adjacency list 
edges = vcat(ports_adj, s_t_adj, transshipment_adj, penalty_adj, dummy_adj)
;

### Dict of adjacency list

In [ ]:
function get_G(edges)
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

### Dijkstra's shortest path algorithm

In [ ]:
# shortest path from source to destination
function dijkstra(G, source, destination)
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end

# Flowproblem

In [ ]:
# storing paths, costs and commodities of LS-MCFP's basis
current_paths = []
current_costs = []
current_commodities = []
;

### Initializing master problem

In [ ]:
# initial basis of RMP constituting penalty arcs 
for i in 1:length(penalty_adj)
    push!(current_paths, penalty_adj[i][1:2])
    push!(current_costs, penalty_adj[i][3])
    append!(current_commodities, i)
end

# edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (ports_adj)
        push!(i_j, string(edge[1], "|", edge[2]))
    end
end

n_ij = size(i_j, 1) # number of edges
ij = 1:n_ij # range of edges

# initializing parameters
n_k = size(WAF_demand.Origin, 1) # number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # number of services
S = 1:n_s

n_p = size(current_paths, 1) # number of paths
P = 1:n_p

n_mv = size(WAF_fleet, 1) # number of vessels in fleet
v = 1:n_mv # range of vessels in fleet

# initial basis including penalty edges
PK = Matrix{Float64}(undef, n_k, 0) # flow conservation constraint matrix
mvs = Matrix{Float64}(undef, n_mv, 0) # vessel constraint matrix
aij = Matrix{Float64}(undef, n_ij, 0) # arc constraint matrix
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity constraint matrix

# constraint matrices for flow conservation and arcs
for i in P
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(current_paths[i])
        if string(current_paths[i][j-1], "|", current_paths[i][j]) ∈ i_j
            ak[findall(x -> x == string(current_paths[i][j-1], "|", current_paths[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
end

# constraint matrix for vessel capacities on arcs
for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

# constraint matrix for service vessles
for i in services
    mv = zeros(Float64, n_mv)
    mv[findall(x -> x == i[3], WAF_fleet."Vessel class")[1]] = 1 * i[1]
    mvs = [mvs mv]
end

P = 1:length(current_paths)
A = 1:size(aij, 2)

revenue = WAF_demand[:,4]
;

### Master problem

In [ ]:
function column_generation(dual_Y_k, dual_Y_ij, n_ij, n_k, n_p, i_j, P, A)

    append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))

    # initializing feasible paths
    candidate_paths = []
    candidate_costs = []
    candidate_commodities = []


    # cost of shortest path
    cij = []


    # update network w.r.t dual variables
    edges_w_dual = deepcopy(edges)
    for i in 1:length(edges_w_dual)
        edges_w_dual[i][3] = edges_w_dual[i][3] - dual_Y_ij[i]
    end


    # shortest path for each commodity
    for i in 1:size(WAF_demand, 1)
        c = 0
        path, cost = dijkstra(get_G(edges_w_dual), string(WAF_demand.Origin[i],"s"), string(WAF_demand.Destination[i],"t"))
        for j in 2:length(path)
            for edge in edges
                if edge[1] == path[j-1] && edge[2] == path[j]
                    c += edge[3]
                end
            end
        end
        push!(candidate_costs, c)
        push!(candidate_paths, path)
        push!(cij, cost)
        append!(candidate_commodities, i)
    end


    # calculating reduced costs
    reducedcost = cij - dual_Y_k - WAF_demand[:,4]


    # initialize basis to be added to RMP
    P_K = Matrix{Float64}(undef, n_k, 0)
    a_ij = Matrix{Float64}(undef, n_ij, 0)


    # adding columns with negative reduced cost
    cp_r_k = candidate_paths[reducedcost .< 0]
    for i in 1:length(cp_r_k)
        ak = zeros(Float64, n_ij)
        pk = zeros(Float64, n_k)
        for j in 2:length(cp_r_k[i])
            if string(cp_r_k[i][j-1], "|", cp_r_k[i][j]) ∈ i_j
                ak[findall(x -> x == string(cp_r_k[i][j-1], "|", cp_r_k[i][j]), i_j)[1]] = 1
            end
        end
        pk[findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1]] = 1
        a_ij = [a_ij ak]
        P_K = [P_K pk]
        push!(current_paths, cp_r_k[i])
        push!(current_costs, candidate_costs[reducedcost .< 0][i])
        append!(current_commodities, findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1])
    end

    
    return reducedcost, a_ij, P_K
end

In [ ]:
Z = [] # storing objective values
iteration = [] # storing iteration count
primal_x = [] # storing x variables
primal_y = [] # storing y variables
len_x = [] # storing number of paths
len_y = [] # storing number of services
nr_re = [] # storing number of paths with negative reduced cost

while true   

    # solving RMP
    flowModel = Model(HiGHS.Optimizer)
    set_silent(flowModel)
    @variable(flowModel, x[P] >= 0)
    @variable(flowModel, 0 <= y[S] <= 1)
    @objective(flowModel, Min, sum(services_w_transshipment[i][2]*y[i] for i in S) + sum((current_costs[j]-revenue[j])*x[j] for j in P))
    @constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
    @constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == WAF_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
    @constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= WAF_fleet.Quantity[i])
    optimize!(flowModel)

    # append Z of each iteration
    append!(Z, objective_value(flowModel))


    # retrieving dual variables
    dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K] # dual variables for commodities at iteration
    dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij] # dual variables for edges with capacity at iteration

    primal_x = value.(x[P]) # primal values for paths
    primal_y = value.(y[S]) # primal values for services

    # retriev data updated data from column generation
    reducedcost, a_ij, P_K = column_generation(dual_Y_k, dual_Y_ij, n_ij, n_k, n_p, i_j, P, A)

    println(reducedcost)

    # updating constraint matrices and their parameters
    aij = [aij a_ij]
    PK = [PK P_K]
    P = 1:length(current_paths)
    n_p = length(current_paths)
    A = 1:size(aij, 2)

    # updating revenue vector that is subtracted from the cost vector
    revenue = WAF_demand[:,4]
    append!(revenue, [WAF_demand[current_commodities[i],4][1] for i in n_k+1:n_p])

    # storing data for analysis
    primal_list_x = []
    for i in primal_x
        append!(primal_list_x, i)
    end
    append!(len_x, length(primal_list_x[n_k+1:end][primal_list_x[n_k+1:end] .> 10^-5]))
    primal_list_y = []
    for i in primal_y
        append!(primal_list_y, i)
    end
    append!(len_y, length(primal_list_y[primal_list_y.> 10^-5]))
    append!(nr_re, length(reducedcost[reducedcost .< -10^-5]))

    # terminaction criterion of RMP
    if all(reducedcost .>= -10^-5)
        break
    end 

end 

In [ ]:
# solving LS-MCFP
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, y[S], Bin)
@objective(flowModel, Min, sum(services_w_transshipment[i][2]*y[i] for i in S) + sum((current_costs[j]-revenue[j])*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == WAF_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= WAF_fleet.Quantity[i])
optimize!(flowModel)
final_primalX = value.(x[P])
final_primalY = value.(y[S])

# storing data from solution
primal_x = []
for i in final_primalX
    append!(primal_x, i)
end
primal_y = []
for i in final_primalY
    append!(primal_y, i)
end
append!(len_x, length(primal_x[n_k+1:end][primal_x[n_k+1:end] .> 10^-5]))
append!(len_y, length(primal_y[primal_y .> 10^-5]))
append!(Z, objective_value(flowModel))
;

In [ ]:
# lists for appending data from each scenario
all_revenues = []
all_x = []
all_y = []
all_z = []
all_len_x = []
all_len_y = []
all_nr_re = []

In [ ]:
append!(all_revenues, [revenue])
append!(all_x, [primal_x])
append!(all_y, [primal_y])
append!(all_z, [Z])
append!(all_len_x, [len_x])
append!(all_len_y, [len_y])
append!(all_nr_re, [nr_re])

In [ ]:
# value and index of best solution
last_elements = [arr[end] for arr in all_z]
min_value = minimum(last_elements)
min_index = argmin(last_elements)

In [ ]:
# resulting paths of best solution
print([current_paths[n_k:end][all_x[min_index][n_k:end] .> 10^-5] all_x[min_index][n_k:end][all_x[min_index][n_k:end] .> 10^-5]])

In [ ]:
# plot of scenario objective value
plot0 = plot(1:length(all_nr_re[1]), all_nr_re[1], label="Low", xlabel="Iteration", ylabel=L"Paths with negative $\bar{c}_{p}$", legend=:topright, linecolor = RGB(252 / 255, 118 / 255, 52 / 255), latexfonts=true)
title!(L"# paths with negative $\bar{c}_{p}$", latexfonts=true)
plot!(1:length(all_nr_re[2]), all_nr_re[2], label="Mid", linecolor = RGB(47 / 255, 62 / 255, 234/255), latexfonts=true)
plot!(1:length(all_nr_re[3]), all_nr_re[3], label="High", linecolor = RGB(153 / 255, 0, 0), latexfonts=true)

plot(plot0, layout=(3, 1))
#savefig("WAF_reducedcosts.svg")

In [ ]:
# plot of objective values
plot1 = plot(1:length(all_z[1]), all_z[1], label="Low", xlabel="Iteration", ylabel="Objective value", legend=:topright, linecolor = RGB(252 / 255, 118 / 255, 52 / 255), latexfonts=true)
title!(L"Best $Z$: %$(round(min_value; digits = 2))", latexfonts=true, titlefont = (12, "sans-serif"))
plot!(1:length(all_z[2]), all_z[2], label="Mid", linecolor = RGB(47 / 255, 62 / 255, 234/255), latexfonts=true)
plot!(1:length(all_z[3]), all_z[3], label="High", linecolor = RGB(153 / 255, 0, 0), latexfonts=true)

plot(plot1, layout=(3, 1))
#savefig("WAF_results_Z.svg")

In [ ]:
# plot of number of paths in solution
plot2 = plot(1:length(all_len_x[1]), all_len_x[1], label="Low", xlabel="Iteration", ylabel="Number of paths with flow", legend=:topright, linecolor = RGB(252 / 255, 118 / 255, 52 / 255), latexfonts=true)
title!(L"# paths for best $Z$: %$(all_len_x[min_index][end])", latexfonts=true)
plot!(1:length(all_len_x[2]), all_len_x[2], label="Mid", linecolor = RGB(47 / 255, 62 / 255, 234/255), latexfonts=true)
plot!(1:length(all_len_x[3]), all_len_x[3], label="High", linecolor = RGB(153 / 255, 0, 0), latexfonts=true)

plot(plot2, layout=(3, 1))
#savefig("WAF_results_x.svg")

In [ ]:
# plot of number of services in solution
plot3 = plot(1:length(all_len_y[1]), all_len_y[1], label="Low", xlabel="Iteration", ylabel="Number of included services", legend=:topright, linecolor = RGB(252 / 255, 118 / 255, 52 / 255), latexfonts=true)
title!(L"# services for best $Z$: %$(all_len_y[min_index][end])", latexfonts=true)
plot!(1:length(all_len_y[2]), all_len_y[2], label="Mid", linecolor = RGB(47 / 255, 62 / 255, 234/255), latexfonts=true)
plot!(1:length(all_len_y[3]), all_len_y[3], label="High", linecolor = RGB(153 / 255, 0, 0), latexfonts=true)

plot(plot3, layout=(3, 1))
#savefig("WAF_results_y.svg")

In [ ]:
println("WAF")
println("z_low = $(all_z[1][end])")
println("z_mid = $(all_z[2][end])")
println("z_high = $(all_z[3][end])")
println("total_revenue_low = $(dot(all_x[1][n_k+1:end],all_revenues[1][n_k+1:end]))")
println("total_revenue_mid = $(dot(all_x[2][n_k+1:end],all_revenues[2][n_k+1:end]))")
println("total_revenue_high = $(dot(all_x[3][n_k+1:end],all_revenues[3][n_k+1:end]))")
println("transported_cargo_low = $(sum(all_x[1][n_k+1:end][all_x[1][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))")
println("transported_cargo_mid = $(sum(all_x[2][n_k+1:end][all_x[2][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))")
println("transported_cargo_high = $(sum(all_x[3][n_k+1:end][all_x[3][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))")
println("number_paths_w_flow_low = $(length(all_x[1][n_k+1:end][all_x[1][n_k+1:end] .> 10^-5]))")
println("number_paths_w_flow_mid = $(length(all_x[2][n_k+1:end][all_x[2][n_k+1:end] .> 10^-5]))")
println("number_paths_w_flow_high = $(length(all_x[3][n_k+1:end][all_x[3][n_k+1:end] .> 10^-5]))")
println("number_services_low = $(all_len_y[1][end])")
println("number_services_mid = $(all_len_y[2][end])")
println("number_services_high = $(all_len_y[3][end])")

In [ ]:
Results = []
append!(Results, ["z_low = $(all_z[1][end])"])
append!(Results, ["z_mid = $(all_z[2][end])"])
append!(Results, ["z_high = $(all_z[3][end])"])
append!(Results, ["total_revenue_low = $(dot(all_x[1][n_k+1:end],all_revenues[1][n_k+1:end]))"])
append!(Results, ["total_revenue_mid = $(dot(all_x[2][n_k+1:end],all_revenues[2][n_k+1:end]))"])
append!(Results, ["total_revenue_high = $(dot(all_x[3][n_k+1:end],all_revenues[3][n_k+1:end]))"])
append!(Results, ["transported_cargo_low = $(sum(all_x[1][n_k+1:end][all_x[1][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))"])
append!(Results, ["transported_cargo_mid = $(sum(all_x[2][n_k+1:end][all_x[2][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))"])
append!(Results, ["transported_cargo_high = $(sum(all_x[3][n_k+1:end][all_x[3][n_k+1:end] .> 10^-5])/sum(WAF_demand[:,3]))"])
append!(Results, ["number_paths_w_flow_low = $(length(all_x[1][n_k+1:end][all_x[1][n_k+1:end] .> 10^-5]))"])
append!(Results, ["number_paths_w_flow_mid = $(length(all_x[2][n_k+1:end][all_x[2][n_k+1:end] .> 10^-5]))"])
append!(Results, ["number_paths_w_flow_high = $(length(all_x[3][n_k+1:end][all_x[3][n_k+1:end] .> 10^-5]))"])
append!(Results, ["number_services_low = $(all_len_y[1][end])"])
append!(Results, ["number_services_mid = $(all_len_y[2][end])"])
append!(Results, ["number_services_high = $(all_len_y[3][end])"])
append!(Results, ["services = $(services[all_y[min_index] .> 10^-5])"])
append!(Results, ["all_revenues $(all_revenues[1:3])"])
append!(Results, ["all_x $(all_x[1:3])"])
append!(Results, ["all_y $(all_y[1:3])"])
append!(Results, ["all_z $(all_z[1:3])"])
append!(Results, ["all_len_x $(all_len_x[1:3])"])
append!(Results, ["all_len_x $(all_len_y[1:3])"])



# saving data
file_path = "WAF_output.txt"
open(file_path, "w") do io
    for row in eachrow(Results)
        line = join(string.(row), "\t")
        println(io, line)
    end
end